In [1]:
%matplotlib inline
import numpy as np
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
import theano.tensor as T
import theano
import lasagne
import lasagne.nonlinearities as nonlin
import pylab
import pickle
import re
import os
import imageio
import time
from IPython.display import display, clear_output
import PIL

from net.vgg16 import make_network_from_file
from net.process import preprocess, resize_img

ModuleNotFoundError: No module named 'pycocotools'

In [2]:
dataDir='annotations_trainval2017'
dataType='val2017'
annFile='{}/annotations/captions_{}.json'.format(dataDir, dataType)

coco=COCO(annFile)

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


In [3]:
print(len(coco.anns))
display(list(coco.anns.items())[-3:])

25014


[(541200,
  {'caption': 'a glass vase with some flower coming out of it ',
   'id': 541200,
   'image_id': 159282}),
 (262135,
  {'caption': 'A tennis player is taking a swing on a red court.',
   'id': 262135,
   'image_id': 551804}),
 (589821,
  {'caption': 'there is a yellow notebook on a black desk ',
   'id': 589821,
   'image_id': 176446})]

# Build and load VGG16

In [4]:
# net, prob_and_vec = make_network_from_file()

# img2vec

In [5]:
from net.imgs import *
      
# %time imgs_process('annotations_trainval2017/img') # Wall time: 4h 43min 38s

In [6]:
imgs = imgs_load(dataDir)
display(len(imgs))

4991

In [7]:
temp = pickle.load(open('annotations_trainval2017\data\\130.data', 'rb'))
temp = list(temp.items())
print('max =', np.max(temp[10][1][1]))
print('min =', np.min(temp[10][1][1]))

max = 11.9422037352
min = 0.0


# word2vec

In [8]:
from net.word2vec import *

E:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
print(StemmingHelper.stem('learning'))
print(StemmingHelper.original_form('learn'))

TypeError: stem() missing 1 required positional argument: 'word'

In [10]:
w2v = load_w2v(5*10**4)

print(w2v.similarity('woman', 'man'), '\n')
print(list(w2v.vocab)[:10], '\n')
print(w2v.most_similar(positive=[w2v['Learning'] - w2v['learning'] + w2v['dog']]), '\n')
print(w2v.most_similar(positive=[w2v['learning'] - w2v['learn'] + w2v['teach']]))

0.7664012231 

['refurbished', 'criminal', 'vivo', 'integrating', 'illiteracy', 'Exporters', 'mirrors', 'Zoom', 'John_Boehner', 'Eastwood'] 

[('dog', 0.7678031921386719), ('dogs', 0.6555119752883911), ('puppy', 0.6472997069358826), ('pit_bull', 0.6181665658950806), ('Dog', 0.6132033467292786), ('pooch', 0.6008472442626953), ('cat', 0.5838526487350464), ('pup', 0.5834978818893433), ('canines', 0.5686283111572266), ('pet', 0.5643360614776611)] 

[('teaching', 0.791033148765564), ('teach', 0.7432790994644165), ('learning', 0.7312349081039429), ('taught', 0.6442131400108337), ('Teaching', 0.6000475883483887), ('curriculum', 0.5869861841201782), ('teaches', 0.5792646408081055), ('instruction', 0.5679458975791931), ('classroom', 0.5501187443733215), ('instructional', 0.5340907573699951)]


In [11]:
from net.sents_process import *

# sents_process(dict_from_anns(coco.anns), 'annotations_trainval2017')

In [13]:
anno = sents_load(dataDir)

In [15]:
danno = anno

In [16]:
captions = [text[1]['caption'] for text in coco.anns.items()]
display(captions[:5])

['Two mean are playing tennis and both are wearing sunglasses.  ',
 'a couple of people that are playing in a field',
 'A purple and white bus in a parking lot.',
 'a tennis player swinging a racket at a ball',
 'a person attempting a jump with a skateboard']

In [19]:
from notebook.dense.helper import mix_text_vec2vec

vects = {}
for id, val in danno.items():
    for v in val:
        vects[id] = vects.get(id, []) + [mix_text_vec2vec(val)]

ValueError: setting an array element with a sequence

# neural networks

In [ ]:
from net.train import *

# Dense net

Generate X, y

In [ ]:
train_size = 23300

X, y = [], []
for id, data in imgs.items():
    while id[0] == '0':
        id = id[1:]
    try:
        for vec in vects[id]:
            X.append(vec)
            y.append(data[1])
    except Exception as ex: 
        print(ex)

print(len(X))

X_train, y_train, X_val, y_val = X[:train_size], y[:train_size], X[train_size:], y[train_size:]

Net

In [ ]:

import theano.tensor as T
import theanonet = build_dense_net('save_01-11-2017_21-45-26.net')
training_dn = Training(net, 40)
training_dn.set_Xy(X_train, y_train, X_val, y_val)

In [ ]:
#training_dn.training(dataDir)

In [ ]:
# save_net(net['last'], 'save_{}.net'.format('LAST'+get_cur_time()))

# Analysis

In [ ]:
def text2vec(text, w2v):
    vecs = [w2v[word] for word in sent2words(text) if word in w2v]
    return mix_text_vec2vec(vecs)

text2vec('dog run').shape

In [ ]:
def show_img(dataDir, file_name):
    img = imageio.imread(dataDir + '/' + file_name)
    img = PIL.Image.fromarray(img)
    img = img.resize((200,200), PIL.Image.ANTIALIAS)
    plt.imshow(img)
    plt.show()
    
def show_img_by_id(img_id):
    show_img(dataDir + '/img', img_id + '.jpg')

In [ ]:
def get_imgs_by_vec(vec, imgs=imgs, net=net):
    result = [(id, np.linalg.norm(vec - point[1])) for id, point in imgs.items()]
    return result

def show_t2i(text, count=3, net=net):
    vec = net.predict_fun_det([text2vec(text)])[0]
    good_imgs = get_imgs_by_vec(vec)[:count]
    good_imgs.sort(key=lambda x: x[1])
    display(good_imgs)
    for id, _ in good_imgs:
        print(id)
        show_img_by_id(id)

In [ ]:
show_t2i('a person attempting a jump with a skateboard')

In [ ]:
procesed_captions = []
for i, text in enumerate(captions):
    mod = 100
    procesed_captions.append(net.predict_fun_det([text2vec(text)])[0])
    if i % mod == 0:
        print('Processed {}'.format(i))

In [ ]:
def get_texts_by_vec(vec, texts=captions, procesed_texts=procesed_captions):
    result = []
    for text, point in zip(texts, procesed_texts):
        dis = np.linalg.norm(vec - point)
        result.append((text, dis))
    result.sort(key=lambda x: x[1])
    return result

def show_i2t(img_id, count=3):
    show_img_by_id(img_id)
    vec = imgs[img_id][1]
    good_text = get_texts_by_vec(vec)[:count]
    print(*good_text, sep='\n', end='\n\n')

In [ ]:
show_i2t('000000565597', count=4)
show_i2t('000000564280', count=4)
show_i2t('000000554291', count=4)
show_i2t('000000543043', count=4)

In [ ]:
def get_texts_by_text(text, net=net, texts=captions, procesed_texts=procesed_captions):
    vec = net.predict_fun_det([text2vec(text)])[0]
    result = []
    for text, point in zip(texts, procesed_texts):
        dis = np.linalg.norm(vec - point)
        result.append((text, dis))
    result.sort(key=lambda x: x[1])
    return result

def show_t2t(text, count=3):
    print(text)
    good_text = get_texts_by_text(text)[:count]
    print(*good_text, sep='\n', end='\n\n')

In [ ]:
show_t2t('A big dog is running')
show_t2t('Green apple')
show_t2t('donut on the table')

In [ ]:
show_t2i('A purple and white bus in a parking lot.')

In [ ]:
show_t2i('donut on table', count=5)

In [ ]:
show_t2i('cat under door', count=3)

In [ ]:
show_t2i('apples on tree', count=10)

In [ ]:
show_t2i('baby with mother', count=3)

In [ ]:
show_t2i('man with an orange', count=10)

In [ ]:
def distance_text_img(text, img_id):
    vec_1 = net.predict_fun_det([text2vec(text)])[0]
    vec_2 = imgs[img_id][1]
    return np.linalg.norm(vec_1 - vec_2)

display(distance_text_img('man with orange', '000000138115'))
display(distance_text_img('man with orange', '000000234526'))

# video sample

In [ ]:
import process

def frames_load(dir_name):
    frames = {}
    for file_name in os.listdir(dir_name + '/video_data'):
        if '.' in file_name and file_name.split('.')[1] == 'data':
            with open('{}/video_data/{}'.format(dir_name, file_name), 'rb') as fl:
                data = pickle.load(fl)
                frames.update(data)
    return frames

frames = frames_load(dataDir)

In [ ]:
imgs_v = {k: v['fc7'] for k, v in frames.items()}

In [ ]:
len(frames)

In [ ]:
video_name = 'Africa.mp4'
vd = imageio.get_reader(video_name)

In [ ]:
def show_video_by_moment(moment, vd=vd):
    img = vd.get_data(moment)
    img = PIL.Image.fromarray(img)
    img = img.resize((200,200), PIL.Image.ANTIALIAS)
    plt.imshow(img)
    plt.show()

In [ ]:
def show_video_t2i(text, count=3, net=net):
    vec = net.predict_fun_det([text2vec(text)])[0]
    good_imgs = get_imgs_by_vec(vec, imgs=imgs_v)[:count]
    good_imgs.sort(key=lambda x: x[1])
    display(good_imgs)
    for id, _ in good_imgs:
        print(id)
        show_video_by_moment(id)

In [ ]:
import scipy.interpolate as interpolate

def show_video_t2graph(text, net=net):
    vec = net.predict_fun_det([text2vec(text)])[0]
    good_imgs = get_imgs_by_vec(vec, imgs=imgs_v)
    good_imgs.sort(key=lambda x: x[0])
    x, y = np.transpose(good_imgs)
    print(y[:6])
    print(len(y))
    k = 1
    x = x[:k + 1] + x
    y = interpolate.BSpline(2.5, x, y, k)
    plt.plot(y)
    plt.show()

In [ ]:
show_video_t2graph('tiger in forest')

In [ ]:
show_video_t2i('tiger in forest', count=10)

# Clusterization img and text

In [ ]:
import sklearn.manifold as manifold
import sklearn.decomposition as decomposition
import 

def show_manifold(mani, imgs, vects, n=200, net=net):
    list_imgs = list(imgs.items())
    perm = np.arange(len(list_imgs))
    np.random.shuffle(perm)
    points = []
    colors = []
    for i in perm[:n]:
        if np.random.randint(0, 2) == 0:
            colors.append(1)
            points.append(list_imgs[i][1][1])
        else:
            colors.append(0)
            id = list_imgs[i][0]
            while id[0] == '0':
                id = id[1:]
            vec = net.predict_fun_det([vects[id][1]])[0]
            points.append(vec)
    
    c_vec = mani.fit_transform(points)
    plt.scatter(*(c_vec).T, c=colors, alpha=0.5, s=50, linewidths=0)
    plt.show()
    
show_manifold(manifold.TSNE(), imgs, vects)
show_manifold(decomposition.PCA(n_components=2), imgs, vects)

In [5]:
import lasagne
lasagne.__version__

'0.1'